# Bank Loan Report

In [ ]:
-- MSSQL could not support date format for time-based columns. So I had to import them as varchar and then convert them to date.
-- I made a stored procedure to convert them to date format in soreted_procedures.sql file so that I can use them in the queries.
-- Converr multiple varchar columns to date columns

--EXEC sp_ConvertMultipleVarcharToDateValidated 'bank_loan_df', 'last_payment_date,next_payment_date,issue_date,last_credit_pull_date', 105,0,0;


In [ ]:
-- Let's check the types of columns to make sure they are correct

SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'bank_loan_df';


### Note: The dataset just contains 2021 data. So I assume:

- Current Date: 31-12-2021

## **A. Summary**

### **Let's get the KPI's**

In [9]:
-- Total loan Applications
SELECT COUNT(id) AS Total_Applications FROM bank_loan_df;

(1 row affected)

Total execution time: 00:00:00.043

Total_Applications
38576


In [10]:
-- MTD (Month-To-Date) Loan Applications
SELECT COUNT(id) AS Total_Applications FROM bank_loan_df
WHERE MONTH(issue_date) = 12 -- AND YEAR(issue_date) = 2021
;

(1 row affected)

Total execution time: 00:00:00.018

Total_Applications
4314


In [18]:
-- PMTD (Previous Month-To-Date) Loan Applications
SELECT COUNT(id) AS Total_Applications FROM bank_loan_df
WHERE MONTH(issue_date) = 11

(1 row affected)

Total execution time: 00:00:00.049

Total_Applications
4035


In [20]:
-- Total Amoun Received
SELECT SUM(total_payment) AS Total_Amount_Collected FROM bank_loan_df

(1 row affected)

Total execution time: 00:00:00.031

Total_Amount_Collected
473070933


In [23]:
-- MTD Total Amount Received
SELECT SUM(total_payment) AS Total_Amount_Collected FROM bank_loan_df
WHERE MONTH(issue_date) = 12

(1 row affected)

Total execution time: 00:00:00.059

Total_Amount_Collected
58074380


In [22]:
-- PMTD Total Amount Received
SELECT SUM(total_payment) AS Total_Amount_Collected FROM bank_loan_df
WHERE MONTH(issue_date) = 11

(1 row affected)

Total execution time: 00:00:00.055

Total_Amount_Collected
50132030


In [24]:
-- Average Interest Rate
SELECT AVG(int_rate)*100 AS Avg_Int_Rate FROM bank_loan_df;

(1 row affected)

Total execution time: 00:00:00.063

Avg_Int_Rate
12.048831397761875


In [26]:
-- MTD Average Interest Rate
SELECT AVG(int_rate)*100 AS MTD_Avg_Int_Rate FROM bank_loan_df
WHERE MONTH(issue_date) = 12

(1 row affected)

Total execution time: 00:00:00.046

MTD_Avg_Int_Rate
12.35604079740363


In [27]:
-- PMTD Average Interest Rate
SELECT AVG(int_rate)*100 AS PMTD_Avg_Int_Rate FROM bank_loan_df
WHERE MONTH(issue_date) = 11

(1 row affected)

Total execution time: 00:00:00.046

PMTD_Avg_Int_Rate
11.941717472118764


In [28]:
-- Average Debt-to-Income Ratio (DTI) for all loans
SELECT AVG(dti)*100 AS Avg_DTI FROM bank_loan_df


(1 row affected)

Total execution time: 00:00:00.041

Avg_DTI
13.327433119037712


In [29]:
-- MTD Avg DTI
SELECT AVG(dti)*100 AS MTD_Avg_DTI FROM bank_loan_df
WHERE MONTH(issue_date) = 12

(1 row affected)

Total execution time: 00:00:00.025

MTD_Avg_DTI
13.665537783959245


In [4]:
-- PMTD Avg DTI
SELECT AVG(dti)*100 AS PMTD_Avg_DTI FROM bank_loan_df
WHERE MONTH(issue_date) = 11

(1 row affected)

Total execution time: 00:00:00.024

PMTD_Avg_DTI
13.30273358116481


### **Loan Status Metrics**

In [6]:
select distinct loan_status from bank_loan_df

(3 rows affected)

Total execution time: 00:00:00.093

loan_status
Fully Paid
Charged Off
Current


In [17]:
SELECT
        loan_status,
        COUNT(id) AS LoanCount,
        SUM(total_payment) AS Total_Amount_Received,
        SUM(loan_amount) AS Total_Funded_Amount,
        AVG(int_rate * 100) AS Interest_Rate,
        AVG(dti * 100) AS DTI
    FROM
        bank_loan_df
    GROUP BY
        loan_status

(3 rows affected)

Total execution time: 00:00:00.060

loan_status,LoanCount,Total_Amount_Received,Total_Funded_Amount,Interest_Rate,DTI
Fully Paid,32145,411586256,351358350,11.641070773060042,13.167350754394208
Charged Off,5333,37284763,65532225,13.878574910931833,14.004732795799702
Current,1098,24199914,18866500,15.099326047358753,14.724344262295066


In [18]:
SELECT 
	loan_status, 
	SUM(total_payment) AS MTD_Total_Amount_Received, 
	SUM(loan_amount) AS MTD_Total_Funded_Amount 
FROM bank_loan_df
WHERE MONTH(issue_date) = 12 
GROUP BY loan_status


(3 rows affected)

Total execution time: 00:00:00.048

loan_status,MTD_Total_Amount_Received,MTD_Total_Funded_Amount
Fully Paid,47815851,41302025
Charged Off,5324211,8732775
Current,4934318,3946625


### **Good Loan VS Bad Loan**

Now let's Let's develop key metrics to compare **charged-off** loans (bad loan) against those that are **fully paid** or **current** (good loan)

In [16]:
-- Good Loan Percentage
SELECT
    (COUNT(CASE WHEN loan_status = 'Fully Paid' OR loan_status = 'Current' THEN id END) * 100.0) / 
	COUNT(id) AS Good_Loan_Percentage
FROM bank_loan_df

Warning: Null value is eliminated by an aggregate or other SET operation.

(1 row affected)

Total execution time: 00:00:00.033

Good_Loan_Percentage
86.175342181667


In [9]:
-- Good Loan Applications
SELECT COUNT(id) AS Good_Loan_Applications FROM bank_loan_df
WHERE loan_status = 'Fully Paid' OR loan_status = 'Current'

(1 row affected)

Total execution time: 00:00:00.033

Good_Loan_Applications
33243


In [10]:
-- Good Loan Funded Amount
SELECT SUM(loan_amount) AS Good_Loan_Funded_amount FROM bank_loan_df
WHERE loan_status = 'Fully Paid' OR loan_status = 'Current'

(1 row affected)

Total execution time: 00:00:00.016

Good_Loan_Funded_amount
370224850


In [11]:
-- Good Loan Amount Received
SELECT SUM(total_payment) AS Good_Loan_amount_received FROM bank_loan_df
WHERE loan_status = 'Fully Paid' OR loan_status = 'Current'

(1 row affected)

Total execution time: 00:00:00.032

Good_Loan_amount_received
435786170


In [12]:
-- Bad Loan Percentage
SELECT
    (COUNT(CASE WHEN loan_status = 'Charged Off' THEN id END) * 100.0) / 
	COUNT(id) AS Bad_Loan_Percentage
FROM bank_loan_df

Warning: Null value is eliminated by an aggregate or other SET operation.

(1 row affected)

Total execution time: 00:00:00.026

Bad_Loan_Percentage
13.824657818332


In [13]:
-- Bad Loan Applications
SELECT COUNT(id) AS Bad_Loan_Applications FROM bank_loan_df
WHERE loan_status = 'Charged Off'

(1 row affected)

Total execution time: 00:00:00.017

Bad_Loan_Applications
5333


In [14]:
-- Bad Loan Funded Amount
SELECT SUM(loan_amount) AS Bad_Loan_Funded_amount FROM bank_loan_df
WHERE loan_status = 'Charged Off'

(1 row affected)

Total execution time: 00:00:00.019

Bad_Loan_Funded_amount
65532225


In [15]:
-- Bad Loan Amount Received
SELECT SUM(total_payment) AS Bad_Loan_amount_received FROM bank_loan_df
WHERE loan_status = 'Charged Off'

(1 row affected)

Total execution time: 00:00:00.018

Bad_Loan_amount_received
37284763


## **B. Overview Section**

In [25]:
-- Monthly Trends by issue_date
SELECT 
	MONTH(issue_date) AS Month_Munber, 
	DATENAME(MONTH, issue_date) AS Month_name, 
	COUNT(id) AS Total_Loan_Applications,
	SUM(loan_amount) AS Total_Funded_Amount,
	SUM(total_payment) AS Total_Amount_Received
FROM bank_loan_df
GROUP BY MONTH(issue_date), DATENAME(MONTH, issue_date)
ORDER BY MONTH(issue_date)


(12 rows affected)

Total execution time: 00:00:00.063

Month_Munber,Month_name,Total_Loan_Applications,Total_Funded_Amount,Total_Amount_Received
1,January,2332,25031650,27578836
2,February,2279,24647825,27717745
3,March,2627,28875700,32264400
4,April,2755,29800800,32495533
5,May,2911,31738350,33750523
6,June,3184,34161475,36164533
7,July,3366,35813900,38827220
8,August,3441,38149600,42682218
9,September,3536,40907725,43983948
10,October,3796,44893800,49399567


In [31]:
-- State Breakdown
SELECT Top 5
	address_state AS State, 
	COUNT(id) AS Total_Loan_Applications,
	SUM(loan_amount) AS Total_Funded_Amount,
	SUM(total_payment) AS Total_Amount_Received
FROM bank_loan_df
GROUP BY address_state
ORDER BY Total_Loan_Applications DESC



(5 rows affected)

Total execution time: 00:00:00.041

State,Total_Loan_Applications,Total_Funded_Amount,Total_Amount_Received
CA,6894,78484125,83901234
NY,3701,42077050,46108181
FL,2773,30046125,31601905
TX,2664,31236650,34392715
NJ,1822,21657475,23425159


In [32]:
-- Loan Term Analysis
SELECT 
	term AS Term, 
	COUNT(id) AS Total_Loan_Applications,
	SUM(loan_amount) AS Total_Funded_Amount,
	SUM(total_payment) AS Total_Amount_Received
FROM bank_loan_df
GROUP BY term
ORDER BY term

(2 rows affected)

Total execution time: 00:00:00.099

Term,Total_Loan_Applications,Total_Funded_Amount,Total_Amount_Received
36 months,28237,273041225,294709458
60 months,10339,162715850,178361475


In [42]:
-- Employee Length
SELECT 
	emp_length AS Employee_Length, 
	COUNT(id) AS Total_Loan_Applications,
	SUM(loan_amount) AS Total_Funded_Amount,
	SUM(total_payment) AS Total_Amount_Received
FROM bank_loan_df
GROUP BY emp_length
ORDER BY emp_length

(11 rows affected)

Total execution time: 00:00:00.036

Employee_Length,Total_Loan_Applications,Total_Funded_Amount,Total_Amount_Received
< 1 year,4575,44210625,47545011
1 year,3229,32883125,35498348
10+ years,8870,116115950,125871616
2 years,4382,44967975,49206961
3 years,4088,43937850,47551832
4 years,3428,37600375,40964850
5 years,3273,36973625,40397571
6 years,2228,25612650,27908658
7 years,1772,20811725,22584136
8 years,1476,17558950,19025777


In [38]:
-- Purpose Breakdown
SELECT 
	purpose AS PURPOSE, 
	COUNT(id) AS Total_Loan_Applications,
	SUM(loan_amount) AS Total_Funded_Amount,
	SUM(total_payment) AS Total_Amount_Received
FROM bank_loan_df
GROUP BY purpose
ORDER BY Total_Loan_Applications DESC

(14 rows affected)

Total execution time: 00:00:00.037

PURPOSE,Total_Loan_Applications,Total_Funded_Amount,Total_Amount_Received
Debt consolidation,18214,232459675,253801871
credit card,4998,58885175,65214084
other,3824,31155750,33289676
home improvement,2876,33350775,36380930
major purchase,2110,17251600,18676927
small business,1776,24123100,23814817
car,1497,10223575,11324914
wedding,928,9225800,10266856
medical,667,5533225,5851372
moving,559,3748125,3999899


In [43]:
-- Home Ownership
SELECT 
	home_ownership AS Home_Ownership, 
	COUNT(id) AS Total_Loan_Applications,
	SUM(loan_amount) AS Total_Funded_Amount,
	SUM(total_payment) AS Total_Amount_Received
FROM bank_loan_df
GROUP BY home_ownership
ORDER BY Total_Loan_Applications DESC;

(5 rows affected)

Total execution time: 00:00:00.044

Home_Ownership,Total_Loan_Applications,Total_Funded_Amount,Total_Amount_Received
RENT,18439,185768475,201823056
MORTGAGE,17198,219329150,238474438
OWN,2838,29597675,31729129
OTHER,98,1044975,1025257
NONE,3,16800,19053
